<a href="https://colab.research.google.com/github/Simrawr/leaves/blob/main/leaf_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
print(torch.__version__)

2.0.1+cu118


In [4]:
from torchvision import datasets, transforms, models
data_dir = "data2/"


In [5]:
!unzip data2.zip

Streaming output truncated to the last 5000 lines.
  inflating: data2/val/pepper healthy/2e28e6a0-2e0b-4284-a9f5-74e5dfac142d___JR_HL 8451.JPG  
  inflating: __MACOSX/data2/val/pepper healthy/._2e28e6a0-2e0b-4284-a9f5-74e5dfac142d___JR_HL 8451.JPG  
  inflating: data2/val/pepper healthy/9b929dc5-778f-45ee-a872-b66cbf78a9e7___JR_HL 7937.JPG  
  inflating: __MACOSX/data2/val/pepper healthy/._9b929dc5-778f-45ee-a872-b66cbf78a9e7___JR_HL 7937.JPG  
  inflating: data2/val/pepper healthy/5f8ceca0-9d35-48cf-847a-c5d39a198321___JR_HL 8297.JPG  
  inflating: __MACOSX/data2/val/pepper healthy/._5f8ceca0-9d35-48cf-847a-c5d39a198321___JR_HL 8297.JPG  
  inflating: data2/val/pepper healthy/489667ae-bca0-4e52-819f-2e04be3ae8cd___JR_HL 8726.JPG  
  inflating: __MACOSX/data2/val/pepper healthy/._489667ae-bca0-4e52-819f-2e04be3ae8cd___JR_HL 8726.JPG  
  inflating: data2/val/pepper healthy/10724f1f-17fb-4443-9878-26e31d97e2d9___JR_HL 7585.JPG  
  inflating: __MACOSX/data2/val/pepper healthy/._10724f1f-1

In [6]:
train_data = datasets.ImageFolder(data_dir + '/train')
val_data = datasets.ImageFolder(data_dir + '/val')

In [7]:
train_transforms = transforms.Compose([transforms.RandomRotation(30), #data augumnetation
                                       transforms.RandomResizedCrop(224),#resize
                                       transforms.RandomHorizontalFlip(), #data augumnetation
                                       transforms.ToTensor()])
val_transforms = transforms.Compose([transforms.RandomResizedCrop(224), #resize
                                      transforms.ToTensor()])
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
val_data = datasets.ImageFolder(data_dir + '/val', transform=val_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=8)
dataiter = iter(train_loader)
images, classes  = next(dataiter)
print(type(images))
print(images.shape)
print(classes.shape)

<class 'torch.Tensor'>
torch.Size([8, 3, 224, 224])
torch.Size([8])


In [8]:
import torch.nn as nn
import numpy as np
class CropDetectCNN(nn.Module):
    # initialize the class and the parameters
    def __init__(self):
        super(CropDetectCNN, self).__init__()

        # convolutional layer 1 & max pool layer 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3),
            nn.MaxPool2d(kernel_size=2))

        # convolutional layer 2 & max pool layer 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1, stride=2),
            nn.MaxPool2d(kernel_size=2))

        #Fully connected layer
        self.fc = nn.Linear(32*28*28, 39)

    # Feed forward the network
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
model = CropDetectCNN()
print(model)

CropDetectCNN(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=25088, out_features=39, bias=True)
)


In [9]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
epochs = 10 # run more iterations
if torch.cuda.is_available():
  model.cuda()

for epoch in range(epochs):
    running_loss = 0
    for images, classes in train_loader:
        # To device - to transfrom the image and classes to CPU|GPU
        images, classes = images.to(device), classes.to(device)

        # clears old gradients from the last step
        optimizer.zero_grad()

        # train the images

        outputs = model(images)


        #calculate the loss given the outputs and the classes
        loss = criterion(outputs, classes)

        # compute the loss of every parameter
        loss.backward()

        # apply the optimizer and its parameters
        optimizer.step()

        #update the loss
        running_loss += loss.item()

    else:
        validation_loss = 0
        accuracy = 0

        # to make the model run faster we are using the gradients on the train
        with torch.no_grad():
            # specify that this is validation and not training
            model.eval()
            for images, classes in val_loader:
                # Use GPU
                images, classes = images.to(device), classes.to(device)

                # validate the images
                outputs = model(images)

                # compute validation loss
                loss = criterion(outputs, classes)

                #update loss
                validation_loss += loss.item()

                # get the exponential of the outputs
                ps = torch.exp(outputs)

                #Returns the k largest elements of the given input tensor along a given dimension.
                top_p, top_class = ps.topk(1, dim=1)

                # reshape the tensor
                equals = top_class == classes.view(*top_class.shape)

                # calculate the accuracy.
                accuracy += torch.mean(equals.type(torch.FloatTensor))

        # change the mode to train for the next epochs
        model.train()
        print("Epoch: {}/{}.. ".format(epoch+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Valid Loss: {:.3f}.. ".format(validation_loss/len(val_loader)),
              "Valid Accuracy: {:.3f}".format(accuracy/len(val_loader)))

RuntimeError: ignored

In [12]:
print(torch.cuda.is_available())

True


In [13]:
model.class_to_idx = train_data.class_to_idx
model.class_to_idx.items()

dict_items([('apple Cassava', 0), ('apple black rot', 1), ('apple scab', 2), ('cedar apple rust', 3), ('cherry healthy', 4), ('cherry powdery mildew', 5), ('corn cercospora leaf spot gray leaf spot', 6), ('corn common rust', 7), ('corn healthy', 8), ('corn northern leaf blight', 9), ('grape black rot', 10), ('grape esca (black measles)', 11), ('grape healthy', 12), ('grape leaf blight (isariopsis leaf spot)', 13), ('healthy', 14), ('orange haunglongbing (citrus greening)', 15), ('peach bacterial spot', 16), ('peach healthy', 17), ('pepper bacterial spot', 18), ('pepper healthy', 19), ('potato early blight', 20), ('potato healthy', 21), ('potato late blight', 22), ('rice BrownSpot', 23), ('rice Healthy', 24), ('rice Hispa', 25), ('rice LeafBlast', 26), ('squash powdery mildew', 27), ('strawberry healthy', 28), ('strawberry leaf scorch', 29), ('tomato bacterial spot', 30), ('tomato early blight', 31), ('tomato healthy', 32), ('tomato late blight', 33), ('tomato leaf mold', 34), ('tomato 